#### change the variables in the following code cell to generate your own plate ref frame grids
- __raster_and_time__
    * A list of mantle frame grids and their reconstruction times. The grids will be converted to plate reference frame acoording to the reconstruction time.
    
- __out_dir__
    * Where you want to save the converted plate reference frame grids.
    
- __multipoint_feature_collection_name__
    * The file name which contains global mesh points. If you are using other rotatoin model, you need to reassign plate ids to the multipoint. Open lat_lon_velocity_domain_720_1440.shp with GPlates and go to "Features->Assign Plate IDs...".
    
- __rotation_filenames__
    * The rotation file name(s)


In [ ]:
import glob

raster_and_time=[
    ('./rasters/M7/MantleFrame/M7.0.Ma.nc', 0),
    ('./rasters/M7/MantleFrame/M7.69.Ma.nc', 69),
    ('./rasters/M7/MantleFrame/M7.140.Ma.nc', 140)
]

out_dir = './rasters/M7/PlateFrame/'
multipoint_feature_collection_name = \
'./reconstructions/Global_Model_WD_Internal_Release_2015_v2/domain_points/lat_lon_velocity_domain_720_1440.shp'
rotation_filenames = glob.glob('./reconstructions/Global_Model_WD_Internal_Release_2015_v2/*.rot')


### The code cell below convert the grids from mantle reference frame to plate reference frame.

In [ ]:
import pygplates
import sys, os
import PlateFrameRasters

if not os.path.isdir(out_dir):
    os.system('mkdir --parents ' + out_dir)
  
time_list = [x[1] for x in raster_and_time]
raster_filenames = [x[0] for x in raster_and_time]

#load the mesh points
file_registry = pygplates.FeatureCollectionFileFormatRegistry()
multipoint_feature_collection = file_registry.read(multipoint_feature_collection_name)
#get rid of points with plate id 0(invalid)
selected_features = multipoint_feature_collection.get(
    lambda feature: feature.get_reconstruction_plate_id()!=0,
    pygplates.FeatureReturn.all)
multipoint_feature_collection = pygplates.FeatureCollection(selected_features)

rotation_model = pygplates.RotationModel(rotation_filenames)

# To Get Plate Frame Grids 
PlateFrameRasters.GeneratePlateReferenceFramesXYZ(
    rotation_model, 
    time_list, 
    raster_filenames, 
    multipoint_feature_collection,
    out_dir)

print('done')


#### The code cell below is for test purpose only. It will draw a netcdf grid. 
Change the __InputGridFile__ variable to the file you want to see.

In [ ]:
import numpy as np
from netCDF4 import Dataset
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
%matplotlib inline

#change this variable to the file you want to see.
#InputGridFile = raster_filenames[0]
InputGridFile = './rasters/M7/PlateFrame/PlateFrameGrid69.00.nc'

def cmap_discretize(cmap, N):
    #http://wiki.scipy.org/Cookbook/Matplotlib/ColormapTransformations
    """Return a discrete colormap from the continuous colormap cmap.     
    cmap: colormap instance, eg. cm.jet. 
    N: number of colors.
    
    Example
    x = resize(arange(100), (5,100))
    djet = cmap_discretize(cm.jet, 5)
    imshow(x, cmap=djet)
    """
    import matplotlib
    
    if type(cmap) == str:
        cmap = get_cmap(cmap)
    colors_i = np.concatenate((np.linspace(0, 1., N), (0.,0.,0.,0.)))
    colors_rgba = cmap(colors_i)
    indices = np.linspace(0, 1., N+1)
    cdict = {}
    for ki,key in enumerate(('red','green','blue')):
        cdict[key] = [ (indices[i], colors_rgba[i-1,ki], colors_rgba[i,ki]) for i in range(N+1) ]
    
    # Return colormap object.
    return matplotlib.colors.LinearSegmentedColormap(cmap.name + "_%d"%N, cdict, 1024)


data = Dataset(InputGridFile,'r')
gridX,gridY = np.meshgrid(data.variables['lon'][:],data.variables['lat'][:])
gridZ = data.variables['z'][:]

# create figure and axes instances
fig = plt.figure(figsize=(16,12),dpi=300,frameon=False)
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
cm = cmap_discretize(plt.cm.RdBu_r, 30)
cb=ax.imshow(gridZ, origin='lower', transform=ccrs.PlateCarree(),
          extent=[-180, 180, -90, 90], cmap=cm)

fig.colorbar(cb, shrink=0.5, label='Dynamic Topography (m)', orientation="horizontal", pad=0.05)
plt.title('Dynamic Topography ')
plt.show()

